# Testing Inference time for QA with Flan T5 Large

In [1]:
%pip install torch transformers sentencepiece -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
OCR_TEXT = """
RENEWAL LEASE FORM Owners and Tenants should read INSTRUCTIONS TO OWNER and INSTRUCTIONS TO TENANT on reverse side before filling out or signing this form THIS IS A NOTICE FOR RENEWAL OF LEASE AND RENEWAL LEASE FORM ISSUED UNDER SECTION 2523.5(a) OF THE RENT STABILIZATION CODE. ALL COPIES OF THIS FORM MUST BE SIGNED BELOW AND RETURNED TO YOUR LANDLORD WITHIN 60 DAYS.
Dated: MARCH 9, 20 22
Tenant's Name(s) and Address: Jorge PizARRO DIANA PRADO, JEORGE PIZARRO
& MICHAEL MOLINA
Owner's /Agent's Name and Address:
MICHAEL FALCO
8 OAK LANE DOUGLASTON, N.Y. 11363
/1
104-50 102nd STREET APT: 2-B OZONE PARK, NEW YORK 11417
1. The owner hereby notifies you that your lease
will expire on: MAY /31, /2022
EXCELSIOR
2. You may renew this lease, for one or two years, at your option, as follows:
Column A Renewal Term
Column B Legal Rent on Sept.30th Preceding Commencement Date of this Renewal Lease
Column C Guideline % or Minimum $ Amount (If unknown, check box and see below)* :unselected:
Column D Applicable Guideline Supplement, if any
Column E Lawful Rent Increase, if any, Effective after Sept. 30th
Column F New Legal Rent (If a lower rent is to be charged, check box and see item 5 below) :unselected:
1 Year
$ 1,461.04
( 1 %) $ 21.91
$
$
$ 1,482.95
2 Years
Same as above
( 22 %) $ 36.52
$
$
$ 1,497.56
* If applicable guideline rate is unknown at time offer is made, check box in Column C and enter current guideline which will be subject to adjustment when rates are ordered.
Additional Deposit Required - 1 year lease: $ 18. 16 Additional Deposit Required - 2 year lease: $ 30 28
3. Security Deposit:
Current Deposit: $ 1, 211.24
4. Specify separate charges, if applicable:
a. Air conditioner : $
b. Appliances : $
c. 421a (2.2%): $ $
d. Other:
Total separate charges: $ :selected:
5. Lower Rent to be charged, if any. 1 year lease $1, 229. 40 2 year lease $1, 241. 52Agreement attached: Yes
6. Tenant shall pay a monthly rent (enter amount from 2F or 5) of $ for a 1 year renewal or $
2 year renewal, plus total separate charges (enter amount from 4) $ for a total monthly payment of $ " for a 1 year renewal or $ for a 2 year renewal. :unselected: No
for a
7. This renewal lease shall commence on JUNE 1, 2022 , which shall not be less than 90 days nor more than 150 days from the date of mailing or personal-delivery of this Renewal Lease Form. This Renewal Lease shall terminate on MAY 31 ,2023 (1 year lease) or MAY 31, 2024 (2 year) lease).
8. This renewal lease is based on the same terms and conditions as your expiring lease. (See instructions about additional provisions.)
9. SCRIE and DRIE. Owner and Tenant acknowledge that, as of the date of this renewal, Tenant is entitled to pay a reduced monthly rent in the amount of $ under the New York City SCRIE program or the New York City DRIE program. The reduced rent may be adjusted by orders of such program. :unselected: 10. Leased premises does
, does not :selected: have an operative sprinkler system. If operative, it was last maintained and inspected
This form becomes a binding lease renewal when signed by the owner below and returned to the tenant. A rider setting forth the rights and obligations of tenants and owners under the Rent Stabilization Law must be attached to this lease when signed by the owner and returned to the tenant. The rent, separate charges and total payment provided for in this renewal lease may be increased or decreased by order or annual updates of the Division of Housing and Community Renewal (DHCR) or the Rent Guidelines Board (RGB).
Tenant: Check and complete where indicated one of three responses below after reading instructions on reverse side. Then date and sign your response below. You must return this Renewal Lease Form to the owner in person or by regular mail, within 60 days of the date this Notice was served upon you by the owner. Your failure to do so may be grounds for the commencement of an action by the owner to evict you from your apartment. :unselected: I (we), the undersigned Tenant(s), accept the offer of a one (1) year renewal lease at a monthly rent of $ :selected: I (we), the undersigned Tenants(s), accept the offer of a two (2) year renewal lease at a monthly rent of $
plus separate charges of $
for a total monthly payment of $
plus separate charges of $
for a total monthly payment of $
1241.52 :unselected: I (we) will not renew my (our) lease and I (we) intend to vacate the apartment on the expiration date of the current lease.
Dated:
3/19
Dated:
3/9
20.22
2022
RTP-8 (9/19)
Tenant's Signature(s):
Stadio
Owner's Signature(s):
Utekaeltolsa :selected: :selected:
"""

In [3]:
# Install necessary libraries if not already installed
# !pip install torch transformers

from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
import torch
import time

# Load the model and tokenizer
model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Use the pipeline for text generation
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=device)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Define a batch of 15 prompts
prompts = [
    f"Q: Given the following context, extract the tenant names:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, extract the complete address of the unit where the tenant resides:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, extract the unit number:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, determine the unit type (e.g., apartment, suite, etc.):\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, extract the agreement date:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, extract the lease start date:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, extract the lease end date:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, identify if the lease has an auto-renewal clause:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, extract the hourly rate, if applicable:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, extract the monthly rent amount:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, identify and extract any prorated rent amounts:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, extract the security deposit amount:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, extract the lawful lease rent amount:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, extract the breakdown of the monthly payment including rent and separate charges:\nContext: {OCR_TEXT}\n A:",
    f"Q: Given the following context, extract any utility charges, if applicable:\nContext: {OCR_TEXT}\n A:"
]

# Start timing the batch inference
start_time = time.time()

# Generate outputs for the batch of prompts using the pipeline
with torch.inference_mode():
    outputs = generator(prompts, max_length=100, truncation=True)

# End timing the inference
end_time = time.time()

# Display the outputs and the time taken for inference
for i, output in enumerate(outputs):
    print(f"Prompt {i+1}: {prompts[i].split('\n')[0]}")
    print(f"Output {i+1}: {output['generated_text']}")
    print()

print(f"Total Inference Time for 15 Prompts: {end_time - start_time:.4f} seconds")


Prompt 1: Q: Given the following context, extract the tenant names:
Output 1: Jorge PizARRO DIANA PRADO, JEORGE PIZARRO & MICHAEL MOLINA

Prompt 2: Q: Given the following context, extract the complete address of the unit where the tenant resides:
Output 2: 8 OAK LANE DOUGLASTON, N.Y. 11363 /1 104-50 102nd STREET APT: 2-B OZONE PARK, NEW YORK 11417

Prompt 3: Q: Given the following context, extract the unit number:
Output 3: 11363

Prompt 4: Q: Given the following context, determine the unit type (e.g., apartment, suite, etc.):
Output 4: Lease

Prompt 5: Q: Given the following context, extract the agreement date:
Output 5: MARCH 9, 20 22

Prompt 6: Q: Given the following context, extract the lease start date:
Output 6: MARCH 9, 20 22

Prompt 7: Q: Given the following context, extract the lease end date:
Output 7: MAY /31, /2022

Prompt 8: Q: Given the following context, identify if the lease has an auto-renewal clause:
Output 8: Yes

Prompt 9: Q: Given the following context, extract the